### Import Required Libraries
---

In [6]:
import os
import csv
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split

### Host or Colab Connection
---

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Debug
print(IN_COLAB)
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

False


In [3]:
if IN_COLAB:
  ds_path = '/content/drive/My Drive/projeto/dataset-article/dataset-64/'
else:
  ds_path = 'data/dataset-source/'

ds_path

'data/dataset-source/'

### Function to generate csv file 
---

In [22]:
def np_zero_image():
    for i, image in enumerate(glob.glob(f"{ds_path}/normal/image/*jpg")):
      # read image in numpy array format
      img = cv2.imread(image)
      
      # zero all pixels values image shape is (w, h, c)
      # convert image to black
      img = np.zeros(img.shape)
      
      # define the path to write the image
      out_path = os.path.join(f"{ds_path}/normal/label/", f"{i+1}.jpg")
      cv2.imwrite(out_path,  img)
        
np_zero_image()

In [4]:
def csv_for_kfold(mode, num_folds, data):
  
  file_path = f"{ds_path}/{mode}{num_folds}.csv"
  fields = ["filepath", "label"]

  write_header = not os.path.exists(file_path)

  with open(file_path, 'a', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=fields)

      if write_header:
        writer.writeheader()

      writer.writerow(data)

In [5]:
relative_path = os.path.relpath('/content/drive/My Drive/projeto/dataset-article/with-stone/', '/content/drive/My Drive/projeto/')
print(relative_path)

dataset-article\with-stone


### Function to create the dataset 
### X: features     y: labels
---

In [6]:
def datasets_gen(ds_path):
  # store relative image file path
  X = []
  # store labels with-stone=1 and without-stone=0
  y = []

  for label_path in os.listdir(ds_path):
    # get labels from images folder
    label = 1 if label_path == 'with-stone' else 0

    # iterate over each file in the dataset
    for image_path in glob.glob(f"{ds_path}/{label_path}/*png"):
      rel_path = os.path.relpath(image_path, ds_path)

      X.append(rel_path)
      y.append(label)

  return X, y

In [7]:
X, y = datasets_gen(ds_path)

### Function to split the dataset into train and validation data
### Train and validation for each kfold
---

In [ ]:
def datasets_gen_kfold(X, y, num_folds):
  
  for num_folds in range(num_folds):

    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, stratify=y)

    data = {}

    # csv file for train data
    for feature, label in tqdm(zip(X_train, y_train), desc=f'Processing data for {num_folds+1} folds'):

      data['filepath'] = feature
      data['label'] = label

      csv_for_kfold(mode='train', num_folds=num_folds, data=data)

    # csv file for validation data
    for feature, label in zip(X_val, y_val):

      data['filepath'] = feature
      data['label'] = label

      csv_for_kfold(mode='val', num_folds=num_folds, data=data)

In [25]:
datasets_gen_kfold(X, y, num_folds=2)

Processing data 1 kfolds: 25it [00:00, 1611.98it/s]
Processing data 2 kfolds: 25it [00:00, 1566.44it/s]
